In [1]:
import os
import csv
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from qiskit_algorithms import VQE
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.optimizers import COBYLA, SLSQP, SPSA
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

### 2 electrons in a 1D infinites square well potential

##### Hamiltonian

In [7]:
optimizer = COBYLA
optimizer_str = 'COBYLA'
L = 1

x = sp.Symbol('x')
x_1 = sp.Symbol('x_1')
x_2 = sp.Symbol('x_2')


def well_wavefunction(n, L, x = sp.Symbol('x')):
    """
    Calcula la función de onda para un estado dado (n) en un pozo de potencial infinito.
    
    Parámetros:
        n (int): Estado cuántico del electrón.
    
    Retorna:
        La función de onda.
    """
    
    # Factor de normalización
    normalization_factor = sp.sqrt(2/L)
    
    # Función de onda radial
    well_wavefunction = normalization_factor * sp.sin(n*sp.pi*x/L)
    
    return well_wavefunction


def hamiltonian_interaction(L, N = 3):
    h_pq = []

    for i in range(1, N+1):
        h_pq.append([])
        for j in range(1, N+1):
            globals()[f'h_{i}{j}'] = -1/2 * sp.integrate(globals()[f'phi_{i}']*globals()[f'dd_phi_{j}'], (x, 0, L))
            h_pq[i-1].append(globals()[f'h_{i}{j}'])

    size = len(h_pq)
    h1_a = np.array(h_pq, dtype=float)
    
    h_ijkl = np.zeros((size, size, size, size))

    for i in range(N):
        for j in range(N):
            for k in range(N):
                for l in range(N):
                    globals()[f'h_{i}{j}{k}{l}'] = globals()[f'phi_{i+1}_1']*globals()[f'phi_{j+1}_2']*\
                                                   globals()[f'phi_{k+1}_2']*globals()[f'phi_{l+1}_1']

                    h_ijkl[i][j][k][l] = sp.integrate(globals()[f'h_{i}{j}{k}{l}'], (x_1, 0, L))

    h2_aa = np.array(h_ijkl, dtype=float)

    hamiltonian = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa)

    return hamiltonian


def hamiltonian_no_interaction(L, N = 3):
    h_pq = []

    for i in range(1, N+1):
        h_pq.append([])
        for j in range(1, N+1):
            globals()[f'h_{i}{j}'] = -1/2 * sp.integrate(globals()[f'phi_{i}']*globals()[f'dd_phi_{j}'], (x, 0, L))
            h_pq[i-1].append(globals()[f'h_{i}{j}'])

    size = len(h_pq)
    h1_a = np.array(h_pq, dtype=float)
    
    h_ijkl = np.zeros((size, size, size, size))
    h2_aa = np.array(h_ijkl, dtype=float)

    hamiltonian = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa)

    return hamiltonian


n_max = 8

for n in range(1, n_max+1):
    for l in range(n):
        globals()[f'phi_{n}'] = well_wavefunction(n, L)
        globals()[f'dd_phi_{n}'] = sp.diff(globals()[f'phi_{n}'], x, x)


n_max = 8

for n in range(1, n_max+1):
    for l in range(n):
        globals()[f'phi_{n}_1'] = well_wavefunction(n, L, sp.Symbol('x_1'))
        globals()[f'phi_{n}_2'] = well_wavefunction(n, L, sp.Symbol('x_1'))

In [9]:
N = 2
hamiltonian = hamiltonian_interaction(L, N)
print(hamiltonian.electronic_integrals.alpha)

Polynomial Tensor
 "+-":
array([[ 4.9348022,  0.       ],
       [ 0.       , 19.7392088]])
 "++--":
array([[[[1.5, 0. ],
         [0. , 1. ]],

        [[0. , 1. ],
         [1. , 0. ]]],


       [[[0. , 1. ],
         [1. , 0. ]],

        [[1. , 0. ],
         [0. , 1.5]]]])


In [12]:
h_pq = hamiltonian.electronic_integrals.alpha['+-']

In [13]:
h_ijkl = hamiltonian.electronic_integrals.alpha['++--']

##### Second quantization formulation

In [28]:
# h_pq = np.array([[9.8696044,         0,         0,         0,         0,          0],
#                  [        0, 24.674011,         0,         0,         0,          0],
#                  [        0,         0, 24.674011,         0,         0,          0],
#                  [        0,         0,         0, 24.674011,         0,          0],
#                  [        0,         0,         0,         0, 24.674011,          0],
#                  [        0,         0,         0,         0,         0, 39.4784176]])

In [4]:
sigma_x = np.array([[0, 1],
                    [1, 0]])

sigma_y = np.array([[ 0, -1j],
                    [1j,   0]])

sigma_z = np.array([[1,  0],
                    [0, -1]])

id = np.array([[1, 0],
               [0, 1]])

Q_plus = np.array([[0, 0],
                   [1, 0]])

Q_minus = np.array([[0, 1],
                    [0, 0]])

In [24]:
H = 0

for p in range(h_pq.shape[0]):
    for q in range(h_pq.shape[1]):

        H += h_pq[p, q] * globals()[f'a_{p}_adjoint'] * globals()[f'a_{q}']

In [29]:
# np.tensordot(sigma_z, sigma_z, axes=0)